In [14]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load Excel file
df = pd.read_csv("mlb_game_data_2025.csv")

# Explore the structure
print(df.shape)
print(df.head(5))

(2463, 21)
             Date Home Team Away Team  Difference doubles  Difference triples  \
0  March 27, 2025       NYY       MIL                 0.0                 0.0   
1  March 27, 2025       TOR       BAL                -1.0                 0.0   
2  March 27, 2025       TEX       BOS                 1.0                 0.0   
3  March 27, 2025       WSH       PHI                -1.0                -1.0   
4  March 27, 2025        KC       CLE                -1.0                -1.0   

   Difference homeruns  Difference hits  Difference hitbypitch  \
0                  1.0              0.0                    0.0   
1                 -5.0            -10.0                   -1.0   
2                 -2.0              1.0                    0.0   
3                 -1.0             -3.0                    1.0   
4                  0.0             -3.0                    0.0   

   Difference avg  Difference obp  ...  Difference ops  \
0           0.029           0.035  ...         

In [15]:
df

,Date,Home Team,Away Team,Difference doubles,Difference triples,Difference homeruns,Difference hits,Difference hitbypitch,Difference avg,Difference obp,...,Difference ops,Difference groundintodoubleplay,Difference leftonbase,Difference strikeouts,Difference baseonballs,Difference whip,Difference strikepercentage,Difference pitchesperinning,Difference homerunsper9,Home Team Won
0,"March 27, 2025",NYY,MIL,0.0,0.0,1.0,0.0,0.0,0.029,0.035,...,0.185,0.0,-3.0,3.0,0.0,-0.16,0.05,-0.06,-1.25,True
1,"March 27, 2025",TOR,BAL,-1.0,0.0,-5.0,-10.0,-1.0,-0.216,-0.171,...,-0.767,2.0,-8.0,5.0,0.0,1.11,0.04,7.11,5.00,False
2,"March 27, 2025",TEX,BOS,1.0,0.0,-2.0,1.0,0.0,0.030,0.007,...,-0.115,1.0,5.0,6.0,1.0,0.00,-0.01,2.00,2.00,False
3,"March 27, 2025",WSH,PHI,-1.0,-1.0,-1.0,-3.0,1.0,-0.064,-0.083,...,-0.296,1.0,0.0,6.0,3.0,0.60,-0.01,2.20,0.90,False
4,"March 27, 2025",KC,CLE,-1.0,-1.0,0.0,-3.0,0.0,-0.078,-0.101,...,-0.258,-1.0,-1.0,-5.0,2.0,0.50,-0.04,1.00,0.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2458,"September 28, 2025",CLE,TEX,0.0,0.0,1.0,4.0,-1.0,-0.008,-0.006,...,-0.014,0.0,0.0,-1.0,-2.0,0.08,0.02,-6.86,-1.10,True
2459,"September 28, 2025",MIA,NYM,2.0,1.0,0.0,4.0,1.0,0.001,-0.012,...,-0.046,-2.0,-5.0,5.0,4.0,-0.02,-0.12,-1.19,0.00,True
2460,"September 28, 2025",MIL,CIN,3.0,0.0,-1.0,2.0,3.0,0.013,0.017,...,0.029,-1.0,15.0,-2.0,-4.0,0.01,0.03,-5.46,0.87,True
2461,"September 28, 2025",ATL,PIT,1.0,0.0,1.0,0.0,0.0,0.014,0.015,...,0.064,0.0,-2.0,4.0,-1.0,0.09,0.00,0.14,-1.13,True


In [16]:
# creates x for features and y for labels
X = df.drop(columns=['Home Team', 'Away Team', 'Date' ,'Home Team Won']).to_numpy()
X = StandardScaler().fit_transform(X)
X


array([[ 0.0151025 , -0.03416992,  0.64186126, ...,  0.82955168,
         0.07054738, -0.70090722],
       [-0.50152899, -0.03416992, -3.05616047, ...,  0.64733588,
         2.19399214,  2.94981181],
       [ 0.531734  , -0.03416992, -1.2071496 , ..., -0.26374313,
         0.68063053,  1.19746667],
       ...,
       [ 1.56499699, -0.03416992, -0.59081265, ...,  0.46512008,
        -1.52869972,  0.53741667],
       [ 0.531734  , -0.03416992,  0.64186126, ..., -0.08152733,
         0.12977875, -0.63081342],
       [-1.01816049, -1.94690863,  0.64186126, ...,  0.46512008,
         0.37855053, -0.63081342]], shape=(2463, 17))

In [17]:
Y = df['Home Team Won'].to_numpy()
Y

array([ True, False, False, ...,  True,  True,  True], shape=(2463,))

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [19]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

for kernel in kernels:
    time_start = time.time()
    clf = svm.SVC(kernel=kernel, C=1.0, gamma='scale')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    time_end = time.time()
    print(f"Training time with {kernel} kernel: {time_end - time_start:.4f} seconds")
    
    accuracy = accuracy_score(y_test, y_pred)
    print (f"Kernel: {kernel}")
    print(f"Accuracy: {accuracy*100:.2f}%")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

Training time with linear kernel: 0.1673 seconds
Kernel: linear
Accuracy: 89.45%
Classification Report:
              precision    recall  f1-score   support

       False       0.86      0.91      0.88       213
        True       0.93      0.89      0.91       280

    accuracy                           0.89       493
   macro avg       0.89      0.90      0.89       493
weighted avg       0.90      0.89      0.89       493

Training time with poly kernel: 0.0905 seconds
Kernel: poly
Accuracy: 87.42%
Classification Report:
              precision    recall  f1-score   support

       False       0.89      0.81      0.85       213
        True       0.87      0.92      0.89       280

    accuracy                           0.87       493
   macro avg       0.88      0.87      0.87       493
weighted avg       0.88      0.87      0.87       493

Training time with rbf kernel: 0.1210 seconds
Kernel: rbf
Accuracy: 88.84%
Classification Report:
              precision    recall  f1-score 